### Анализ тональности отзывов
В качестве выборки возьмем отзывы на фильмы из библиотеки NLTK

In [15]:
from nltk.corpus import movie_reviews

# Возьмем ids негативных и позитивных отзывов
neg_ids = movie_reviews.fileids('neg')
pos_ids = movie_reviews.fileids('pos')

pos_ids[:5]

['pos/cv000_29590.txt',
 'pos/cv001_18431.txt',
 'pos/cv002_15918.txt',
 'pos/cv003_11664.txt',
 'pos/cv004_11636.txt']

Подготовим обучающую выборку (тексты и классы)

In [29]:
# Загружаем негативные и позитивные отзывы, и собираем наш текст отзывов
neg_reviews = [' '.join(movie_reviews.words(fileids=[file])) for file in neg_ids]
pos_reviews = [' '.join(movie_reviews.words(fileids=[file])) for file in pos_ids]

reviews = neg_reviews + pos_reviews

# Создаем метки классов
labels = [0] * len(neg_reviews) + [1] * len(pos_reviews)
print('Total Labels: ', len(labels))

Total Labels:  2000


In [31]:
# Взглянем на один отзыв
reviews[1]

'the happy bastard \' s quick movie review damn that y2k bug . it \' s got a head start in this movie starring jamie lee curtis and another baldwin brother ( william this time ) in a story regarding a crew of a tugboat that comes across a deserted russian tech ship that has a strangeness to it when they kick the power back on . little do they know the power within . . . going for the gore and bringing on a few action sequences here and there , virus still feels very empty , like a movie going for all flash and no substance . we don \' t know why the crew was really out in the middle of nowhere , we don \' t know the origin of what took over the ship ( just that a big pink flashy thing hit the mir ) , and , of course , we don \' t know why donald sutherland is stumbling around drunkenly throughout . here , it \' s just " hey , let \' s chase these people around with some robots " . the acting is below average , even from the likes of curtis . you \' re more likely to get a kick out of h

In [67]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

import warnings

warnings.filterwarnings("ignore")

Функция ниже создана для удобства оценивания различных моделей.

In [40]:
# Pipeline создаст частоты слов, затем на основе частот вычислит TFIDF, полученная матрица признаков затем классифицируется
def model_pipeline(vectorizer, transformer, classifier):
    return Pipeline([
      ('vectorizer', vectorizer),
      ('transformer', transformer),
      ('classifier', classifier)
    ])

Сравним качество различных классификаторов на данных отзывах

In [41]:
for classifier in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(classifier)
    print(cross_val_score(model_pipeline(CountVectorizer(), TfidfTransformer(), classifier()), reviews, labels).mean())
    print()

<class 'sklearn.linear_model._logistic.LogisticRegression'>
0.8205

<class 'sklearn.svm._classes.LinearSVC'>
0.8545

<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>
0.857



В нашем случае оказались хороши 2 модели, возьмем как итоговую SGDClassifier. Обучим его на всех отзывах

In [43]:
# Здесь используем сразу TfidfVectorizer(), он комбинирует в себе CountVectorizer() и TfidfTransformer()

classifier = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', SGDClassifier())
    
])

classifier.fit(reviews, labels)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', SGDClassifier())])

### Пример классификации отзыва
Помним, что 0 - негативный, 1 - положительный. Однако даже учитывая относительно неплохое качество, классификатор можно легко обмануть

In [53]:
print(classifier.predict(['Yesterday I decided to watch a film called the mummy. The film surprised me!']))
print(classifier.predict(['Yesterday I decided to watch a film called the mummy. The film is not as bad as I expected. I had a great time']))

[1]
[0]


Попробуем как-нибудь улучшить качество модели

### Понижение размерности и ансамбли деревьев
Поробуем применить различные матричные разложения

In [54]:
from sklearn.decomposition import NMF, TruncatedSVD

In [59]:
# Найдем для начала матрицу частот слов и взглянем на её размерность
c_vec = CountVectorizer()
feature_matrix = c_vec.fit_transform(reviews)
feature_matrix.shape

(2000, 39659)

Число признаков в 39к действительно много. Попробуем понизить размерность и применить модели посложней, например, деревья.

In [68]:
# Неотрицательное матричное разложение NMF
nmf = NMF(10)

# Понизим размерность до 10 признаков
nmf_feature_matrix = nmf.fit_transform(feature_matrix)

In [62]:
# SVD
svd = TruncatedSVD(10)

# Понизим размерность до 10 признаков
svd_feature_matrix = svd.fit_transform(feature_matrix)

Сравним теперь SVD и NMF используя pipeline

In [73]:
for dec_method in [TruncatedSVD, NMF]:
    print(dec_method)
    print(cross_val_score(model_pipeline(CountVectorizer(), dec_method(n_components=10), LinearSVC()), reviews, labels).mean())

<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>
0.5385000000000001
<class 'sklearn.decomposition._nmf.NMF'>
0.655


Качество явно стало ниже, возможно число компонент равно 10 недостаточно. Посмотрим на результаты с 1000 компонентами. Возьмем TruncatedSVD, т.к. NMF c 1000 компонентами преобразует очень долго. 

In [75]:
%%time
print(cross_val_score(model_pipeline(TfidfVectorizer(),
                               TruncatedSVD(n_components=1000),
                               LinearSVC()), reviews, labels).mean())

0.849
Wall time: 1min 38s


Как видим вы молучили исходное качество, но уже с меньшим числом признаков. Возможно, используя деревья мы сможем улучшить качество?

### Ансамбли деревьев на признаках меньшей размерности

In [79]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [81]:
%%time 
print(cross_val_score(Pipeline([
    ('vectorizer', CountVectorizer()),
    ('transformer', TruncatedSVD(n_components=100)),
    ('classifier', RandomForestClassifier(n_estimators=100))
]), reviews, labels).mean())

0.7335
Wall time: 17 s


Особо не помагло, увеличим до 1000 число деревьев и компонент

In [82]:
print(cross_val_score(Pipeline([
    ('vectorizer', CountVectorizer()),
    ('transformer', TruncatedSVD(n_components=1000)),
    ('classifier', RandomForestClassifier(n_estimators=1000))
]), reviews, labels).mean())

0.7244999999999999


Тоже не помагло, качество даже еще чуть просело. Может нужно вместо частот слов использовать TFIDF?

In [83]:
print(cross_val_score(Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('transformer', TruncatedSVD(n_components=1000)),
    ('classifier', RandomForestClassifier(n_estimators=1000))
]), reviews, labels).mean())

0.631


Качество еще сильней упало. Может необходмо совместить признаки из TFIDF и из SVD разложения? Возможно неплохая идея.

In [84]:
from sklearn.pipeline import FeatureUnion # объединяет преобразования и получает единое множество признаков

# Добавим одну компоненту из SVD разложения 
estimators = [('tfidf', TfidfTransformer()), ('svd', TruncatedSVD(1))]
combined = FeatureUnion(estimators)

In [85]:
print(cross_val_score(Pipeline([
    ('vectorizer', CountVectorizer()),
    ('transformer', combined),
    ('classifier', LinearSVC())
]), reviews, labels).mean())

0.6775


Качество получилось ниже. Таким образом отправной бейзлайн был очень хороший.